In [60]:
import numpy as np
import pandas as pd

In [61]:
sim_types = [
                'Ricker',
                # 'May',
                # 'Rosen-Mac'
            ]

In [62]:
seed = 1234

In [63]:
# Open data sets
data = {sim_type: pd.read_csv(f'../prediction_values/pred_{sim_type}_{seed}.csv').rename(columns={'Unnamed: 0': 'Index'}).set_index('Index') for sim_type in sim_types}

In [64]:
indicators = ['variance', 'cv', 'ac1', 'skew']

In [76]:
def compute_scores(data:dict, sim_type:str, cont_tables:dict):

    df = data[sim_type]
    print(df.head())

    df_scores = df.copy()

    ktau_crit = 0.8

    for indicator in indicators:

        df_scores[indicator] = (np.abs(df[indicator]) >= ktau_crit) * 1

        # Fill in scores in contingency tables
        cont_tables[indicator][0, 0] = df_scores.loc[(df_scores[indicator] == 1) & (df_scores['true'] == 1)].shape[0]
        cont_tables[indicator][0, 1] = df_scores.loc[(df_scores[indicator] == 1) & (df_scores['true'] == 0)].shape[0]
        cont_tables[indicator][1, 0] = df_scores.loc[(df_scores[indicator] == 0) & (df_scores['true'] == 1)].shape[0]
        cont_tables[indicator][1, 1] = df_scores.loc[(df_scores[indicator] == 0) & (df_scores['true'] == 0)].shape[0]
        
        # Normalize scores
        cont_tables[indicator][:, 0] /= df_scores[df_scores['true'] == 1].count()[indicator]
        cont_tables[indicator][:, 1] /= df_scores[df_scores['true'] == 0].count()[indicator]

        print()
        print(indicator)
        print(cont_tables[indicator])

    return cont_tables

cont_tables = {
    indicator: np.zeros((2,2)) for indicator in indicators
}
cont_tables = compute_scores(data, 'Ricker', cont_tables)

       variance        cv       ac1      skew  true  pred_interval_max
Index                                                                 
0      0.804422  0.869048  0.911565  0.615646     1                0.9
1     -0.396825  0.004608 -0.152074 -0.023041     1                0.9
2      0.630769  0.733333  0.766667  0.497436     1                0.9
3      0.808696  0.885990  0.949758  0.818357     1                0.9
4      0.918051  0.951274  0.918051 -0.377630     1                0.9

variance
[[0.38 0.  ]
 [0.62 1.  ]]

cv
[[0.64 0.12]
 [0.36 0.88]]

ac1
[[0.58 0.12]
 [0.42 0.88]]

skew
[[0.04 0.  ]
 [0.96 1.  ]]
